# Schema Silver

Autor: Thiago Vilarinho Lemes\
Projeto: ETL no Databricks utilizando Catalog \
Data: 14/09/2025

In [71]:
from databricks.connect import DatabricksSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum, when
import os
from dotenv import load_dotenv
import pandas as pd

In [72]:
# Carrega as variáveis de ambiente do arquivo .env

load_dotenv()

catalog_name = os.getenv("NAME_CATALOG")

schema_bronze = os.getenv("NAME_SCHEMA_BRONZE")
table_bronze = os.getenv("NAME_TABLE_BRONZE")

schema_silver = os.getenv("NAME_SCHEMA_SILVER")
table_silver = os.getenv("NAME_TABLE_SILVER")

schema_gold = os.getenv("NAME_SCHEMA_GOLD")
table_gold = os.getenv("NAME_TABLE_GOLD")

schema_raw = os.getenv("NAME_SCHEMA_RAW")
bucket_raw = os.getenv("NAME_STORAGE")

In [73]:
# Cria a sessão Spark usando Databricks Connect
spark = DatabricksSession.builder.getOrCreate()

In [89]:
# Seleciona o catalog e schema a serem utilizados
try:
    spark.sql(f"USE CATALOG {catalog_name}")
    spark.sql(f"USE SCHEMA {schema_silver}")
    print(f"Catalog e Schema selecionados: {catalog_name}.{schema_silver}")
except Exception as e:
    print(f"Erro ao selecionar Catalog ou Schema: {e}")

Catalog e Schema selecionados: catalogo_netflix_movies.silver


In [75]:
catalog_name = catalog_name

In [91]:
try:
    df = spark.read.table(f"{catalog_name}.{schema_bronze}.{table_bronze}")
    print(f"Leitura da tabela {catalog_name}.{schema_bronze}.{table_bronze} realizada com sucesso.")
    df.show(5)
except Exception as e:
    print(f"Erro ao ler a tabela: {e}")

Leitura da tabela catalogo_netflix_movies.bronze.None realizada com sucesso.


+-------+-------+--------------------+--------------------+--------------------+--------------------+----------------+------------+------+--------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|      date_added|release_year|rating|duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------------+------------+------+--------+--------------------+--------------------+
|  s6566|  Movie|         Dark Crimes|  Alexandros Avranas|Jim Carrey, Marto...|United Kingdom, P...|October 15, 2019|        2016|     R|  93 min|   Dramas, Thrillers|A detective on a ...|
|  s6567|  Movie|         Dark Places|Gilles Paquet-Bre...|Charlize Theron, ...|United States, Un...|   July 26, 2018|        2015|     R| 113 min|Dramas, Internati...|Years after survi...|
|  s6568|  Movie|      Darna Mana Hai|        Praw

Analise Exploratória

In [82]:
# Nome das colunas
print("Nomes das colunas no DataFrame:")
print(df.columns)

Nomes das colunas no DataFrame:
['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']


In [78]:
# Quantidade de linhas e colunas do DataFrame
print(f"Quantidade de linhas do Dataframe: {df.count()} linhas")
print(f"Quantidade de colunas do Dataframe: {len(df.columns)} colunas")

Quantidade de linhas do Dataframe: 17509 linhas
Quantidade de colunas do Dataframe: 12 colunas


In [ ]:
# Verificando os valores nulos de cada coluna
try:
    nulos_por_coluna = df.select([
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in df.columns
    ])
    print("Verificação de valores nulos realizada com sucesso.")
    nulos_por_coluna.show()
except Exception as e:
    print(f"Erro ao verificar valores nulos: {e}")

Verificação de valores nulos realizada com sucesso.


+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   0|    0|    5172|1642|   1657|        10|           0|     4|       3|        0|          0|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [ ]:
# Verificando os valores únicos da coluna date_added
df_unicos = df.select("date_added").distinct()

# Mostra os valores
print("Valores únicos na coluna date_added:")
df_unicos.show()

Valores únicos na coluna date_added:


+-----------------+
|       date_added|
+-----------------+
|December 31, 2017|
|       2021-05-11|
|       2020-09-20|
|    March 1, 2018|
|       2020-12-28|
|       2020-11-22|
|       2020-08-20|
|February 15, 2018|
|       2021-04-06|
|       2021-02-15|
|       2020-09-02|
|  August 13, 2019|
|December 19, 2017|
|       2020-09-29|
|   March 29, 2019|
|       2020-10-12|
|       2021-09-09|
|       2021-08-12|
|       2021-07-01|
|       2020-10-11|
+-----------------+
only showing top 20 rows


In [ ]:
# Verificando os valores únicos da coluna release_year
df_unicos = df.select("release_year").distinct()

# Mostra os valores
print("Valores únicos na coluna release_year:")
df_unicos.show()

Valores únicos na coluna release_year:


+------------+
|release_year|
+------------+
|        1987|
|        1975|
|        1964|
|        2013|
|        1954|
|        1993|
|        2009|
|        1991|
|        1992|
|        2003|
|        2011|
|        2005|
|        2018|
|        1979|
|        2019|
|        2004|
|        1988|
|        1943|
|        1997|
|        2020|
+------------+
only showing top 20 rows


In [ ]:
# Verificando os valores únicos da coluna type
df_unicos = df.select("type").distinct()

# Mostra os valores
print("Valores únicos na coluna type:")
df_unicos.show()

Valores únicos na coluna type:


+-------+
|   type|
+-------+
|TV Show|
|  Movie|
+-------+



In [ ]:
# Verificando os valores únicos da coluna rating
df_unicos = df.select("rating").distinct()

# Mostra os valores
print("Valores únicos na coluna rating:")
df_unicos.show()

Valores únicos na coluna rating:


+--------+
|  rating|
+--------+
|   TV-14|
|    TV-Y|
|   TV-PG|
|      UR|
|   TV-Y7|
|       G|
|   TV-MA|
|      NR|
|       R|
|      PG|
|    TV-G|
|   PG-13|
|TV-Y7-FV|
|   NC-17|
|    NULL|
|  84 min|
|  74 min|
|  66 min|
+--------+



Normalizando dados

In [92]:
df.show(15)

+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|       date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+---------+--------------------+--------------------+
|  s6566|  Movie|         Dark Crimes|  Alexandros Avranas|Jim Carrey, Marto...|United Kingdom, P...| October 15, 2019|        2016|     R|   93 min|   Dramas, Thrillers|A detective on a ...|
|  s6567|  Movie|         Dark Places|Gilles Paquet-Bre...|Charlize Theron, ...|United States, Un...|    July 26, 2018|        2015|     R|  113 min|Dramas, Internati...|Years after survi...|
|  s6568|  Movie|      Darna Mana Hai|  

In [94]:
# Converte para pandas
# Melhora a manipulação de datas com formatos mistos
try:

    # Converte DataFrame Spark para pandas
    df_pandas = df.toPandas()
    print("- Conversão para pandas realizada com sucesso.")

    # Remove espaços e normaliza valores nulos/vazios
    df_pandas['date_added'] = df_pandas['date_added'].astype(str).str.strip()
    df_pandas['date_added'] = df_pandas['date_added'].replace(['', 'None', 'nan', 'NaT'], 'noIns')
    print("- Limpeza inicial da coluna date_added realizada com sucesso.")

    # Converte para datetime de forma inteligente (mixed format)
    df_pandas['date_added'] = pd.to_datetime(df_pandas['date_added'], format='mixed', errors='coerce')
    print("- Conversão para datetime realizada com sucesso.")

    # Converte para string no formato yyyy-MM-dd, substituindo nulos por "noIns"
    df_pandas['date_added'] = df_pandas['date_added'].dt.strftime('%Y-%m-%d').fillna('noIns')
    print("- Formatação da coluna date_added realizada com sucesso.")

    # Recria o DataFrame do Spark
    df = spark.createDataFrame(df_pandas)
    print("- Conversão da coluna date_added para datetime realizada com sucesso.")
    df.show(5)
except Exception as e:
    print(f"Erro ao converter a coluna date_added: {e}")

- Conversão para pandas realizada com sucesso.
- Limpeza inicial da coluna date_added realizada com sucesso.
- Conversão para datetime realizada com sucesso.
- Formatação da coluna date_added realizada com sucesso.
- Conversão da coluna date_added para datetime realizada com sucesso.
+-------+-------+--------------------+---------------+--------------------+--------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|       country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+--------------+----------+------------+------+---------+--------------------+--------------------+
|  s8755|  Movie|              Wolves|Bart Freundlich|Michael Shannon, ...| United States|2019-03-29|        2016|     R|  109 min|Dramas, Independe...|A promising high ...|
|  s8756|TV Show|  

In [ ]:
# Verificando os valores únicos da coluna date_added após a limpeza
df_unicos = df.select("date_added").distinct()
print("Valores únicos na coluna date_added após a limpeza:")
df_unicos.show(5)

Valores únicos na coluna date_added após a limpeza:


+----------+
|date_added|
+----------+
|2021-05-11|
|2020-01-11|
|2020-09-20|
|2019-11-20|
|2020-12-28|
+----------+
only showing top 5 rows


In [ ]:
# Verificando os valores nulos de cada coluna novamente
print("Verificando os valores nulos da coluna 'date_added':")
df.filter(F.col("date_added") == "noIns").show(5)

Verificando os valores nulos da coluna date_added:
+-------+-------+-------------------+--------+--------------------+-------------+----------+------------+------+----------+--------------------+--------------------+
|show_id|   type|              title|director|                cast|      country|date_added|release_year|rating|  duration|           listed_in|         description|
+-------+-------+-------------------+--------+--------------------+-------------+----------+------------+------+----------+--------------------+--------------------+
|  s6796|TV Show|            Frasier|    NULL|Kelsey Grammer, J...|United States|     noIns|        2003| TV-PG|11 Seasons|Classic & Cult TV...|Frasier Crane is ...|
|  s6807|TV Show|            Friends|    NULL|Jennifer Aniston,...|United States|     noIns|        2003| TV-14|10 Seasons|Classic & Cult TV...|This hit sitcom f...|
|  s6902|TV Show|    Gunslinger Girl|    NULL|Yuuka Nanri, Kana...|        Japan|     noIns|        2008| TV-14| 2 Seas

In [ ]:
# Normalizando dados
# Substituindo valores nulos ou vazios por 'notIns'
try:
    df_clean = df
    for c in df.columns:
        df_clean = df_clean.withColumn(
            c,
            F.when(
                F.col(c).isNull() | (F.trim(F.col(c)) == ""),  # nulo ou vazio
                F.lit("notIns")
            ).otherwise(F.col(c))
        )
    print("- Normalização de dados realizada com sucesso.")
    df_clean.show(5)
except Exception as e:
    print(f"Erro ao normalizar dados: {e}")

- Normalização de dados realizada com sucesso.
+-------+-------+--------------------+---------------+--------------------+--------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|       country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+--------------+----------+------------+------+---------+--------------------+--------------------+
|  s8755|  Movie|              Wolves|Bart Freundlich|Michael Shannon, ...| United States|2019-03-29|        2016|     R|  109 min|Dramas, Independe...|A promising high ...|
|  s8756|TV Show|   Women Behind Bars|         notIns|              notIns| United States|2016-11-01|        2010| TV-14|3 Seasons|Crime TV Shows, D...|This reality seri...|
|  s8757|  Movie|           Woodstock|  Barak Goodman|              notIns| United 

In [ ]:
# Verificando os valores nulos de cada coluna
try:
    nulos_por_coluna = df_clean.select([
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in df_clean.columns
    ])
    print("Verificação de valores nulos após a normalização.")
    nulos_por_coluna.show()
except Exception as e:
    print(f"Erro ao verificar valores nulos: {e}")

Verificação de valores nulos após a normalização realizada com sucesso.


+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   0|    0|       0|   0|      0|         0|           0|     0|       0|        0|          0|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|      0|   0|    0|       0|   0|      0|         0|           0|     0|       0|        0|          0|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [63]:
try:
    df_clean.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.{table_silver}")
    print("DataFrames carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar DataFrames: {e}")

DataFrames carregados com sucesso!


In [99]:
# Carregando os dados da tabela silver para verificação
df_silver = spark.read.table(f"{catalog_name}.{schema_silver}.{table_silver}")
df_silver.show(5)

+-------+-------+--------------------+----------------+--------------------+--------------------+----------+------------+------+--------+--------------------+--------------------+
|show_id|   type|               title|        director|                cast|             country|date_added|release_year|rating|duration|           listed_in|         description|
+-------+-------+--------------------+----------------+--------------------+--------------------+----------+------------+------+--------+--------------------+--------------------+
|  s8754|  Movie|           Withdrawn|   Adrian Murray|Aaron Keogh, Moll...|              Canada|2018-05-01|        2017| TV-MA|  74 min|Dramas, Independe...|Unable to pay bil...|
|  s4378|  Movie|Trevor Noah: Son ...|David Paul Meyer|         Trevor Noah|       United States|2018-11-20|        2018| TV-MA|  64 min|     Stand-Up Comedy|"Daily Show" host...|
|  s4379|  Movie|     The Pixar Story|   Leslie Iwerks|         Stacy Keach|       United States|201